In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


In [15]:
# Endogenous (dependent): What you want to model
endog_cols = [
    'LNG 174K CBM (2-stroke dual fuel) Spot Rate\n(avg., $/day)',
    'LNG Capacity (CBM)',
    'Spread\nEU-US', 'Spread\nASIA-US', 'Spread\nASIA-EU'
]
# Exogenous (independent): External factors
exog_cols = [
    'Global price of Natural Gas, Asia (start month, MMBTU)',
    'Global price of Natural Gas, EU (start month, MMBTU)',
    'Global price of Natural Gas, US Henry Hub (start month, MMBTU)'
]

# Reload full dataset (before column drops)
data = pd.read_csv("datatest_3.csv")
data['Date'] = pd.to_datetime(data['Date'], format="%b-%Y")
data.set_index('Date', inplace=True)
data = data.apply(pd.to_numeric, errors='coerce')
data.dropna(inplace=True)

# Split
endog = data[endog_cols]
exog = data[exog_cols]


In [16]:
for df in [endog, exog]:
    for col in df.columns:
        if adfuller(df[col])[1] > 0.05:
            df[col] = df[col].diff()
    df.dropna(inplace=True)

endog, exog = endog.align(exog, join='inner', axis=0)


/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_14902/1918674293.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].diff()
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_14902/1918674293.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].diff()
/var/folders/q_/5f9r9hrn7fgg50f9b9171j940000gn/T/ipykernel_14902/1918674293.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [17]:
endog_scaler = StandardScaler()
exog_scaler = StandardScaler()

endog_scaled = pd.DataFrame(endog_scaler.fit_transform(endog), index=endog.index, columns=endog.columns)
exog_scaled = pd.DataFrame(exog_scaler.fit_transform(exog), index=exog.index, columns=exog.columns)


In [18]:
train_size = int(len(endog_scaled) * 0.9)

endog_train, endog_test = endog_scaled.iloc[:train_size], endog_scaled.iloc[train_size:]
exog_train, exog_test = exog_scaled.iloc[:train_size], exog_scaled.iloc[train_size:]


In [23]:
model = VAR(endog_train)
model_fit = model.fit(maxlags=5, exog=exog_train)  # Try different lags if needed


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


TypeError: VAR.fit() got an unexpected keyword argument 'exog'

In [20]:
lag_order = model_fit.k_ar
forecast_input = endog_train.values[-lag_order:]
forecast_exog = exog_test.values

forecast_scaled = model_fit.forecast(y=forecast_input, steps=len(exog_test), exog_future=forecast_exog)

# Inverse transform the forecast
forecast = pd.DataFrame(endog_scaler.inverse_transform(forecast_scaled), index=endog_test.index, columns=endog.columns)


NameError: name 'model_fit' is not defined

In [21]:
from sklearn.metrics import mean_squared_error

endog_test_original = pd.DataFrame(endog_scaler.inverse_transform(endog_test), index=endog_test.index, columns=endog.columns)

for col in endog.columns:
    mse = mean_squared_error(endog_test_original[col], forecast[col])
    print(f"{col} MSE: {mse:.2f}")


NameError: name 'forecast' is not defined